# **MAE 598: Machine Learning for Engineers**
# **Lane Detection**
## Group Members
### * Aksheshkumar Ajaykumar Shah - 1217506160
### * Poojan Chetanbhai Makwana - 1217717124
### * Sai Karthik Mysore Mouliswar - 1217135829
### * Sharanya Parameshwar Hebbar - 1217410207 

# Step 1: Importing Important Libraries
### * In this step we will be importing the important libraries that will be helping us ahead to develop the model by using thier built-in functions
### * The libraries include OpenCV to process the video, Numpy to perform mathematical operations, Matplot library to plot figures.

In [3]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


# **Step 2: Development of the helper functions.**
### These helper functions are developed by using the built in functions of the above mentioned libraries. These helper functions will be called in our model to help us detect lanes on the video. The helper function do_canny performs operations like doing canny transformations of each frame we obtain from the video, the do_segment function is divinding the image in segments and determing the segment that is important to us, the houghs functions is used to apply hough tranformation to our frame and highligh the lane with green light, the calculate_coordinates functions id used to calculate the coordinates of the line that are supposed to be marked on our frames to show lane, calculate_line function works on finding the line from the coordinate we have developed from the previous function, Visualize_lines functions is used to visualize the hough lines on our frame in the video to show the lane.

In [4]:
def do_canny(frame):
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    blur = cv.GaussianBlur(gray, (5,5),0)
    cann = cv.Canny(blur,50,150)
    return cann


def do_segment(canny):
    height = canny.shape[0]
    polygons = np.array([[(0,height),(800,height),(380,290)]])
    mask = np.zeros_like(canny)
    cv.fillPoly(mask,polygons,255)
    segment = cv.bitwise_and(canny,mask)
    return segment

def houghs(segment):
    hough = cv.HoughLinesP(segment,2,np.pi/180,100,np.array([]),minLineLength = 100, maxLineGap = 50)
    return hough


def calculate_coordinates(frame,par):
    slope, intercept = par
    y1 = frame.shape[0]
    y2 = int(y1-150)
    x1 = int((y1-intercept)/slope)
    x2 = int ((y2-intercept)/slope)
    coordinates = np.array([x1,y1,x2,y2])
    return coordinates

def calculate_lines(frame,hough):
    left = []
    right = []
    for line in hough:
        x1,y1,x2,y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2),(y1,y2),1)
        slope = parameters[0]
        y_int = parameters[1]
        if slope<0:
            left.append((slope,y_int))
        else:
            right.append((slope,y_int))
        if right:
            right_avg = np.average(right,axis=0)
            right_line = calculate_coordinates(frame,right_avg)
        if left:
            left_avg = np.average(left,axis=0)
            left_line = calculate_coordinates(frame, left_avg)
    return np.array([left_line,right_line])




def visualize_lines(frame,lines):
    lines_visualize = np.zeros_like(frame)
    if lines is not None:
        for x1,y1,x2,y2 in lines:
            cv.line(lines_visualize,(x1,y1),(x2,y2),(0,255,0),5)
    return lines_visualize









# Step 3: Creating a Model
## In this step we will be recalling the above developed helper functions in order to detect lane on the data. The video is first read using the openCV function VideoCapture and the it is split into frames per 10 milisecond. these frames are feed to canny edge detector in order for it to find the lane markings on the road. Later hough Transformatioon is used to plot the lane and the plotted lane is imposed on the normal frame.

In [5]:
def Model(file_path):#input is in the form of video
    cap = cv.VideoCapture(file_path)
    while (cap.isOpened()):
        ret,frame = cap.read() # frame if the current frame of the video and ret is the boolean value from getting the frame.
        if ret:
            canny = do_canny(frame)
            cv.imshow("Canny Transformation",canny)
            segment = do_segment(canny)
            hough = houghs(segment)
            lines = calculate_lines(frame,hough)
            lines_visualize = visualize_lines(frame, lines)
            cv.imshow("Hough Transformation",lines_visualize)
            output = cv.addWeighted(frame,0.9,lines_visualize,1,1)
            cv.imshow("Final Result",output)
# we will be taking frames in the intervals of 10 seconds each and the key q can be used to close the program anytime we want.
            if cv.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv.destroyAllWindows()
    return output

# **Step 4: Calling the model**
## In this step we feed in the video input to our model for it to process the video and give us the desired output that is the lanes on the road.

In [6]:
file_path = r'C:\Users\shaha\Documents\Arizona State University\MAE 598 Machine Learning\project\input.mp4'
Model(file_path)